In [ ]:
import numpy as np
from sympy import *
import sympy
from sympy.utilities.lambdify import lambdify
from sympy import symbols, Eq, solve, collect
import random as rand
import random
import IPython.display as disp
init_printing()
import matplotlib.pyplot as plt
from numpy.random import default_rng
import heapq

rng = default_rng(seed=42)

In [ ]:
# Order of the equation
n = 3

# y-type terms
T = n + 2

min_subterms = 2
max_subterms = 3
max_fun = 3

# Cumulative probabilities for different function types
function_cdf = [0.1, 0.8, 0.62, 0.4, 1, 1]

add_c_prob = 0.5

In [ ]:
x = Symbol('x')

y_terms = np.empty(T, dtype=Symbol)
y_terms[0] = x

for i in range(1, T):
    y_terms[i] = Symbol('y'+str(i-1))
    
y_terms

### Functions

In [ ]:
def num_subterms (sum_min, sum_max):
    temp = np.empty(T, dtype=int)
    temp[-1] = 1

    for i in range(T - 1):
        probabilities = [0] * 1 + [1] * 2
        temp[i] = rand.choice(probabilities)

    if (sum(temp) < sum_min or sum_min > sum_max):
        return num_subterms (sum_min, sum_max)

    return [1, 1, 1, 1, 1] 

In [ ]:
def add_coeff_and_constant(subterm, const_prob = 0.5):
    coeffs = np.linspace(-5,-1,11, dtype=int)
    if rand.random() < 0.5:
        subterm *= rand.choice(coeffs)

    if rand.random() < const_prob:
        subterm += rand.choice(coeffs)

    return subterm

In [ ]:
def generate_random_sympy_polynomial():
    x = symbols('x')
    num_terms = random.choice([2, 3])
    polynomial = 0

    for _ in range(num_terms):
        coeff = random.choice([i for i in range(-5, 6) if i != 0])
        power = random.randint(0, 4)
        polynomial += coeff * x**power
    
    return polynomial

random_sympy_polynomial = generate_random_sympy_polynomial()

In [ ]:
def generate_random_sympy_polynomial():
    trig_functions = [sin, cos, tan]  
    chosen_function = random.choice(trig_functions)

    x = symbols('x')
    num_terms = random.choice([1, 2])
    powers = random.sample(range(0, 3), num_terms)  
    polynomial = 0

    if rand.random() > 0.5:
        for power in powers:
            coeff = random.choice([i for i in range(-3, 4) if i != 0])
            polynomial += coeff * x**power
    else:
        return chosen_function(x)+rand.randint(1, 5)

    return polynomial


In [ ]:
def create_subterm(y_term, cdf, iteration, max_it, const_prob):

    subterm = y_term

    if y_term == y_terms[4]:
        return y_terms[4]

    temp = rand.random()

    if y_term == y_terms[2]:
        power = rand.randint(2, 5)
        if power == 3:
                power += random.choice([-1, 1])
        return y_term ** power

    if y_term in [y_terms[1], y_terms[3]]:
        if rand.random() > 0.75: 
            power = rand.randint(2, 5)  
            if power == 3:
                power += random.choice([-1, 1]) 
            return y_term ** power
        else:
            polynomial = generate_random_sympy_polynomial()
            subterm = y_term / polynomial
            return simplify(subterm)

    if y_term == y_terms[0]:
        if temp > 0.75:
            trig_fun = [sin(y_term)] + [cos(y_term)]
            polynomial = generate_random_sympy_polynomial()
            subterm = 1/polynomial
            return simplify(subterm)
        else:
            subterm = add_coeff_and_constant(y_term, const_prob)
            return simplify(subterm)

    if iteration < max_it:
        return create_subterm(subterm, cdf, iteration + 1, max_it, const_prob)

    return simplify(subterm)

In [ ]:
def create_subterms (y_term, num_sub):
    subterms = np.empty(sum(num_sub), dtype=Mul)

    counter = 0

    for i, y_t in enumerate(y_term):
        for n_s in range(num_sub[i]):
            subterms[counter] = create_subterm(y_t, function_cdf, 1, max_fun, add_c_prob)
            counter += 1

    return subterms

In [ ]:
terms = create_subterms(y_terms, num_subterms(min_subterms, max_subterms))
y3 = y_terms[4]
new_terms = terms.tolist()
new_terms.remove(y3)
new_terms

y3 = -sum(new_terms)

### Generating the ODE

In [ ]:
from sympy import symbols, diff, exp
from scipy.integrate import solve_ivp

x = symbols('x')
y, y0, y1, y2, y3 = symbols('y y0 y1 y2 y3')

terms = create_subterms(y_terms, num_subterms(min_subterms, max_subterms))

replacement_mapping = {
    y0: y,
    y1: symbols("y'"),
    y2: symbols("y''"),
    y3: 0
}

def replace_with_differential(expr):
    return expr.subs(replacement_mapping)

replaced_expr = replace_with_differential(sum(terms))

latex_replaced_expr = sympy.latex(replaced_expr)

a = random.randint(0, 3)
b = random.randint(0, 3)
c = random.randint(0, 3)

updated_latex_equation = f"""Consider the following third-order ordinary differential equation $$y''' = {latex_replaced_expr}$$
with initial conditions at $x = 0: y(0) = {a:.2f}, y'(0) = {b:.2f}, y''(0) = {c:.2f}.$
Find analytical expressions that approximate the solution of $y(x)$ in the small and large $x$ regimes.
"""

y3 = y_terms[4]
new_terms = terms.tolist()
new_terms.remove(y3)
y3 = sum(new_terms)
print("y3 is " + str(y3))
print(y3)

y3_func = lambdify((x, y0, y1, y2), y3)

def system_of_odes_with_y3(x, y):
    y0, y1, y2 = y
    d3y_dx3 = y3_func(x, y0, y1, y2)
    return [y1, y2, d3y_dx3]

initial_conditions = [a, b, c]  
x_range = (0.001, 100)
X = np.linspace(x_range[0], x_range[1], 10000)

solution_with_y3 = solve_ivp(system_of_odes_with_y3, x_range, initial_conditions, method='DOP853', rtol=1e-3, atol=1e-6, t_eval=X)

plt.plot(solution_with_y3.t, solution_with_y3.y[0], mfc='none', label='Numerical Solution')
plt.legend()
plt.xlabel('x')
plt.ylabel('y(x)')
plt.title('Numerical Solution')
plt.grid(True)
plt.show()

### Interpolated solution for downstream numerical evaluations

In [ ]:
y3_func = lambdify((x, y0, y1, y2), y3)

def system_of_odes_with_y3(x, y):
    y0, y1, y2 = y
    # Use y3 as a function
    d3y_dx3 = y3_func(x, y0, y1, y2)
    return [y1, y2, d3y_dx3]

initial_conditions = [a,b,c] 
x_range = (0.001, 100)

solution_with_y3_dense = solve_ivp(system_of_odes_with_y3, x_range, initial_conditions, method='DOP853', rtol=1e-3, atol=1e-6, dense_output=True)

plt.plot(solution_with_y3_dense.t, solution_with_y3_dense.y[0], mfc='none', label='Numerical Solution')
plt.legend()
plt.xlabel('x')
plt.ylabel('y(x)')
plt.title('Numerical Solution')
plt.grid(True)
plt.show()

### Plotting the dominant balances

In [ ]:
x, y0, y1, y2 = symbols('x y0 y1 y2')

new_terms = terms[:4]
print(new_terms)

term_funcs = [lambdify((x, y0, y1, y2), new_term) for new_term in new_terms]

x_values = solution_with_y3.t
y_values0 = solution_with_y3.y[0]
y_values1 = solution_with_y3.y[1]
y_values2 = solution_with_y3.y[2]

# Evaluate each function with the numerical solutions
evaluated_terms = [func(x_values, y_values0, y_values1, y_values2) for func in term_funcs]

y_values3 = sum(evaluated_terms)

term1, term2, term3, term4 = evaluated_terms
list_of_terms = [term1, term2, term3, term4]
term5 = y_values3

In [ ]:
for i in range(len(list_of_terms)):
    plt.plot(solution_with_y3.t, abs(list_of_terms[i]), label = f"Term {i+1}");

plt.plot(solution_with_y3.t, abs(term5), label = "Term 5");
plt.xlabel('x')
plt.ylabel('magnitude')
plt.xlim([0,solution_with_y3.t[-1]])
plt.yscale("log")
plt.title("Comparing magnitudes of each term at small x")
plt.legend();

In [ ]:
div_point = solution_with_y3.t[-1]

### Approximate Formula for Divergent Solution

In [ ]:
y, x = symbols('y x')

def translate_to_differential(expr):
    return expr.subs({y0: y, y1: Derivative(y, x), y2: Derivative(y, x, x)})

last_x_value = solution_with_y3.t[-1]
evaluated_at_last_x = [abs(term[-1]) for term in list_of_terms]
terms_with_index = [(i, value) for i, value in enumerate(evaluated_at_last_x)]
terms_with_index.sort(key=lambda x: x[1], reverse=True)
largest_terms_indices = [terms_with_index[0][0]]

largest_term_index = largest_terms_indices[0]
largest_term_expression = new_terms[largest_term_index] if largest_term_index < len(new_terms) else None

largest_term_differential = translate_to_differential(largest_term_expression) \
if largest_term_expression is not None else None

y_triple_prime_equation = Eq(Derivative(y, x, x, x), largest_term_differential)

y_triple_prime_equation

### Plug in the Ansatz to Obtain Approximation

In [ ]:
alpha, p = symbols('alpha p')
x = symbols('x')

y_ansatz = alpha * (div_point - x)**(p)
y_prime_expr = -alpha * p * (div_point - x)**(p - 1)
y_double_prime_expr = alpha * p * (p - 1) * (div_point - x)**(p - 2)
y_triple_prime_expr = -alpha * p * (p - 1) * (p - 2) * (div_point - x)**(p - 3)

equation_with_derivatives = y_triple_prime_equation.subs({
    Derivative(y, x): y_prime_expr,
    Derivative(y, x, x): y_double_prime_expr,
    Derivative(y, x, x, x): y_triple_prime_expr
})

equation_with_derivatives

### Finding $\alpha$ and $p$

In [ ]:
lhs_collected = collect(equation_with_derivatives.lhs, (div_point-x))
rhs_collected = collect(equation_with_derivatives.rhs, (div_point-x))

coeffs1, power1 = lhs_collected.as_coeff_exponent((div_point-x))
coeffs2, power2 = rhs_collected.as_coeff_exponent((div_point-x))

coeff_eq = Eq(coeffs1, coeffs2)
power_eq = Eq(power1, power2)

# Solving the system of equations
solution = solve((coeff_eq, power_eq), (alpha, p))

if not solution:
    raise Exception("No solution")

for sol in solution:
    alpha_val, p_val = sol
    if alpha_val != 0 and p_val != 0:
        nonzero_solution = sol
        break
    
if alpha_val == 0 or p_val == 0:
    raise Exception("No nonzero solution found for p and alpha")


### Plot

In [ ]:
def analyticalapproximation_init(x):
    return alpha_val * (div_point - x) ** p_val

In [ ]:
modified_t_values = solution_with_y3.t[:-1]

# Calculating the constant C to fit the ansatz
fit_constant = a - analyticalapproximation_init(modified_t_values[0])

def analyticalapproximation(x):
    return alpha_val * (div_point - x) ** p_val + fit_constant

Yana_init = analyticalapproximation_init(modified_t_values)
Yana = analyticalapproximation(modified_t_values)
fit_constant = round(fit_constant.evalf(), 2)

In [ ]:
plt.figure(figsize=[14, 10])
plt.plot((modified_t_values), solution_with_y3.y[0,:-1], '-', label='Numerical')
plt.plot((modified_t_values), Yana, 'o', label='Analytical with fit')
plt.title("Solution Comparison")
plt.xlabel("$x$")
plt.ylabel("y")
plt.legend()
plt.show()

End the notebook if the solution at big $x$ is not within 10% of the numerical solution

In [ ]:
numerical_sol_large_x = solution_with_y3_dense.sol(solution_with_y3.t[:-1][-5])[0]
percent_error = np.abs((Yana[-5]-numerical_sol_large_x) / numerical_sol_large_x).evalf() * 100

print(f"Percent error is: {percent_error}")
if percent_error > 10:
    raise Exception("Solution inaccurate")

For the small x regime, the dominant terms are determined by looking at the point div_point / 4 (arbitrarily chosen, but to prevent it from going into other intermediate regimes) 

### Small $x$ Taylor series

In [ ]:
def small_x_sol(terms):
    sum = 0
    for i, term in enumerate(new_terms):
        if i == 0:
            term = term.subs(x, 0)
        if i == 1:
            term = term.subs(y0, a)
        if i == 2:
            term = term.subs(y1, b)
        if i == 3:
            term = term.subs(y2, c)
        sum += term
    taylor_expansion = a + b*x + c/2*x**2 + sum/6 * x**3
    return taylor_expansion

In [ ]:
small_x_sol(new_terms)

In [ ]:
def analytical_small(val):
    approx = small_x_sol(new_terms)
    return approx.subs(x, val)

### Visual verification for both regimes

In [ ]:
small_x_analytical = small_x_sol(new_terms)

modified_t_values = solution_with_y3.t[:-1]

# Calculate the analytical approximation using the modified t values
Yana_large = analyticalapproximation(modified_t_values)
Yana_small = lambdify(x, small_x_analytical)

end_index_smallx = int(2/5*len(modified_t_values))
end_index_largex = int(4/5*len(modified_t_values))

# Plotting
plt.figure(figsize=[14, 10])
plt.plot((modified_t_values), solution_with_y3.y[0,:-1], '-', label='Numerical')
plt.plot((modified_t_values[end_index_largex:-1]), Yana_large[end_index_largex:-1], 'o', label='Analytical for large x')
plt.plot((modified_t_values[:end_index_smallx]), Yana_small(modified_t_values[:end_index_smallx]), 'o', label='Analytical for small x')


plt.title("Solution Comparison")
plt.xlabel("$x$")
plt.ylabel("y")
plt.legend()
plt.show()

### Formatting for the data generator

In [ ]:
x = sympy.symbols('x')
y = sympy.Function('y')(x)

derivatives = [y, y.diff(x), y.diff(x, x), y.diff(x, x, x)]

first_term_replaced = terms[0].subs(x, x) # x
second_term_replaced = terms[1].subs(y0, y) # y
third_term_replaced = terms[2].subs(y1, y.diff(x)) 
fourth_term_replaced = terms[3].subs(y2, y.diff(x, x)) 
fifth_term_replaced = terms[4].subs(y3, y.diff(x, x, x)) 

sum_terms = first_term_replaced + second_term_replaced + third_term_replaced + fourth_term_replaced

differential_equation = Eq(fifth_term_replaced, sum_terms)

dollar_sign = "$$"

print(f"The third-order ODE is given by the following equation: " + "$$" + latex(differential_equation) + "$$"
    + " and has initial conditions, $y(0)={a}, y'(0)={b}, y''(0)={c}. Using the method of dominant balance, \
        find analytical solutions that approximate \
        the solution to the ODE at (a) very small x and (b) large x.")



### Large x evaluation point

In [ ]:
large_x_eval = 0.99 * div_point
print(large_x_eval)

### Approximate value at small x solution

In [ ]:
small_x_eval = 0.1
print(analytical_small(small_x_eval))

### Approximate value at large x solution

In [ ]:
y_at_large_x = analyticalapproximation(large_x_eval).evalf()
print(y_at_large_x)

### Numerical value at small x

In [ ]:
print(solution_with_y3_dense.sol(small_x_eval)[0])

if np.abs((analytical_small(small_x_eval) - solution_with_y3_dense.sol(small_x_eval)[0]) / solution_with_y3_dense.sol(small_x_eval)[0]) * 100 > 10:
    raise Exception("Solution Inaccurate")

### Numerical value at large x

In [ ]:
print(solution_with_y3_dense.sol(large_x_eval)[0])

### Small eps solution

In [ ]:
print("y = " + latex(small_x_sol(new_terms)))

### Large eps solution

In [ ]:
large_solution_latex = r"""
y = %s(%s - x)^{%s}+%s 

""" % (latex(alpha_val), latex(round(div_point, 2)), latex(p_val), latex(fit_constant))

print(large_solution_latex)

### Both solutions boxed

In [ ]:
large_x_latex_print = large_solution_latex.replace("\n", " ").strip()

full_latex_bracketless = "$$\\boxed{y = " + latex(small_x_sol(new_terms)) + ", " + large_x_latex_print + "}$$"


full_latex_string = "$$\\boxed{[y = " + latex(small_x_sol(new_terms)) + ", " + large_x_latex_print + "]}$$"
print(full_latex_string)

### Problem Statement again

In [ ]:
print(updated_latex_equation)

**Solution**

In [ ]:
solution_latex = r"""
The solution in the small $x$ regime can be approximated using a Taylor series up to the third order. Plugging in the provided initial conditions, the Taylor series is:
$$y = %s.$$ 

To find the solution in the large $x$ regime, the method of dominant can be used. The dominant balance is given by $$%s.$$

A power law ansatz can be used to approximate the solution at the divergence point. The ansatz has the form 
$$y = \alpha (x^* - x)^p,$$ 
where $x^*$ is the divergence point and can be determined through numerical code. Plugging in the ansatz and solving for the terms yields 
$$%s$$

After substituting the derivatives, the equation is reorganized to collect terms with respect to $(x^* - x)$. This leads to an equation where the coefficients and powers of $(x^* - x)$ are equated on both sides. Simplifying the equation gives us two separate equations, one for the coefficients and another for the powers of $(x^* - x)$.

The coefficients' equation is:
$$%s.$$

The powers' equation is:
$$%s.$$

Solving this system of equations provides the values of $\alpha$ and $p$. 
A valid solution is identified if $\alpha$ and $p$ are both nonzero. 
In this case, the solution for $\alpha$ and $p$ is found to be:
$$\alpha = %s, \quad p = %s.$$

The ansatz is now $$y' = %s(%s-x)^{%s},$$ but to account for the initial conditions, a constant, $c=a-y'(0)$, is calculated and added to the ansatz.
Thus, an analytical solution that approximates the solution at large $x$ is given by
$$y = %s(%s-x)^{%s} + %s.$$


Therefore, the full solution is given by 
%s
""" % (latex(small_x_sol(new_terms)), latex(y_triple_prime_equation), latex(equation_with_derivatives), latex(coeff_eq), latex(power_eq), 
       latex(alpha_val), latex(p_val),
       latex(alpha_val), latex(div_point), latex(p_val),
       latex(alpha_val), latex(div_point), latex(p_val), latex(fit_constant),
       full_latex_bracketless
       )

print(solution_latex)